### Simulando a coleta de dados com Spark Streaming através da "escuta" em uma porta TCP-IP especificada
#### Para isso, iremos usar o netcat como ferramenta de apoio.
porta escolhida: 22121

comando: nc -lk 22121

## 1. Importando os módulos necessários para o Streaming de Dados

In [ ]:
#fonte: https://github.com/danielsan/Spark-Streaming-Examples/blob/master/spark-streaming-foreachRDD-and-foreach.py
# Módulos do Spark

from pyspark.streaming import StreamingContext
from pyspark import SparkContext # quando usamos o PYSPARK, o SPARK CONTEXT já é criado por default: sc

## 2. Criando o contexto com o Spark Streaming
- Lembrando que o contexto com a aplicação Spark, por default pelo PYSPARK já é criado automaticamente com o nome "sc"

In [ ]:
print('-->> Verificando o contexto em que se encontra a conexão:', sc) #sc = spark context
print('-->> Versão do SPARK em execução:', sc.version)

# Definindo o contexto do Streaming de dados com Spark, uma vez que o contexto com o Spark já foi criado por default
strcontext = StreamingContext(sparkContext = sc, batchDuration = 1)

## 3. Criando o RECEIVER do Spark.
- No caso, estamos usando o socketTextStream por se tratar de uma conexão à uma porta TCP-IP
- A coleta de dados é possível através do Twitter, Apache Flume, Apache Kafka, HDFS do Hadoop, IOT: ou seja, as fontes de dados para o RECEIVER que irá "alimentar" o Streaming do Spark. Veja, são inúmeras.

In [ ]:
# Criando o RECEIVER para fazer o streaming de dados TCP-IP = socketTextStream
hostname = "localhost"
port = 22121

lines = strcontext.socketTextStream(hostname = hostname, port = port)
print("Type object 'lines':", lines)

## 4. Tratamento e Tranformação
### 4.1. Para cada linha, divide as palavras a cada " " (espaço) encontrado

In [ ]:
# como estamos executando função de transforamção sobre o DSTREAM gerado (lines), então devemos "jogar" o resultado
# da transformação em um novo DSTREAM, pois este é sempre IMUTÁVEL.
words = lines.flatMap(lambda lines : lines.split(" "))

print("Type object 'words':", words)

### 4.2. Conta o número de ocorrências das palavras em cada batch entregue pelo streaming de dados

In [ ]:
pairs = words.map(lambda words : (words, 1))
# Exemplo de saída: (('ciência', 1), ('Big Data', 2), ('abacaxi', 1))

wordCounts = pairs.reduceByKey(lambda x, y: x + y) # onde a chave é a própria palavra!
print("Type object 'wordCounts':", type(wordCounts))

## 5. Imprimindo os 10 primeiros elementos de cada RDD gerado no DStream
RDD = Resilient Distributed Dataset

In [ ]:
wordCounts.pprint()

## 6. Funções para as etapas de:
- Criação do Receiver para o Streaming de dados.
- Tranformação dos dados ainda no DStream
- Persistência dos dados do DStream para Tupla

In [ ]:
import pandas as pd

# Módulos do Spark
from pyspark.streaming import StreamingContext
#from pyspark import SparkContext # quando usamos o PYSPARK, o SPARK CONTEXT já é criado por default: sc

print('-->> Verificando o contexto em que se encontra a conexão:', sc) #sc = spark context
print('-->> Versão do SPARK em execução:', sc.version)

# Definindo o contexto do Streaming de dados com Spark, uma vez que o contexto com o Spark já foi criado por default
strcontext = StreamingContext(sparkContext = sc, batchDuration = 10)

hostname = "localhost"
port = 22121

# Criando uma lista vazia
values = list()

def ReceiverDataStreaming(hostname, port):
    '''
    Especificação da função...
    '''
    # Criando o RECEIVER no Streaming Context
    lines = strcontext.socketTextStream(hostname = hostname, port = port)
    return(lines)

def TransformationDataStreaming():
    '''
    Especificação da função...
    '''
    # Chamando a função para construção do RECEIVER
    lines = ReceiverDataStreaming(hostname = hostname, port = port)

    # Para cada linha, divide as palavras a cada " " (espaço) encontrado
    words = lines.flatMap(lambda lines : lines.split(" "))
    # 'words' seria algo como = "abacaxi abacaxi Data " => [('abacaxi', 'Data', 'abacaxi', '')]

    # Conta o número de ocorrências das palavras em cada batch entregue pelo streaming de dados
    pairs = words.map(lambda words : (words, 1))
    # 'pairs' seria algo como: [('abacaxi', 1), ('abacaxi', 1), ('Data', 1), ('abacaxi', 1), ('', 1)]
    
    wordsCount = pairs.reduceByKey(lambda x, y: x + y) # onde a chave é a própria palavra!
    # 'wordsCount' seria algo como: [('abacaxi', 3), ('Data', 1), ('', 1)]
    return(wordsCount)

def SendRecord(tup):
    '''
    Especificação da função...
    '''
    word   = tup[0]
    amount = tup[1]
    content = (word, amount)
    values.append(content)
    #print(values)
    # Aqui poderia ser inserido os valores dentro de um banco de dados MongoDB.

def PersistDSTream(DSTREAM):
    '''
    Objetivo: persistir os dados do DSTREAM em uma tupla.
    
    A parâmetro de entrada será sempre o microbatching gerado pelo DSTREAM (que nada mais é que uma micro coleção
    de dados (RDD)!), onde é gerado pela função TransformationDataStreaming().
    '''
    DSTREAM.foreachRDD(lambda rdd_values : rdd_values.foreach(SendRecord))
    return(DSTREAM.pprint())

### 6.1. Executando as funções declaradas...

In [ ]:
PersistDSTream(DSTREAM = TransformationDataStreaming())

## 7. Início e encerramento da coleta do stream de dados

- strcontext.start() = Iniciando a coleta e processamento do stream de dados.
- strcontext.awaitTermination() = a coleta de dados por streaming irá rodar indefinidamente até que encontre um erro de execução ou caso finalize todo o trabalho de streaming de dados.

In [ ]:
strcontext.start()
strcontext.awaitTermination()

### 7.1 Encerrando o RECEIVER...

In [ ]:
strcontext.stop()